# Basics of deep learning and neural networks


In [4]:
import numpy as np
from typing import NewType, Dict

In [6]:
NumpyArray = NewType('NumpyArray', np.ndarray)

## Section 1. Basics of deep learning and neural networks

We will code a **forward propagation** (prediction) for our first neural network.

Say that each data point is a customer. The first input is how many accounts they have, and the second input is how many children they have. The model will predict **how many transactions the user makes in the next year**. 

In [14]:
# Hard-code input data and weights
num_accounts: int = 5
num_children: int = 3

input_data: NumpyArray = np.array([num_accounts, num_children])
    
weights: Dict[str, NumpyArray] = {'node_0': np.array([2, 4]), 
                                  'node_1': np.array([ 4, -5]), 
                                  'output': np.array([2, 7])}

# Calculate the nodes and the output of the neural network
node_0_value = (input_data * weights['node_0']).sum()

node_1_value = (input_data * weights['node_1']).sum()

hidden_layer_outputs = np.array([node_0_value, node_1_value])

output: np.int32 = (hidden_layer_outputs * weights['output']).sum()

output

79

It looks like the network generated a prediction of $79$.